#  Going Deeper On Molecular Featurizations

One of the most important steps of doing machine learning on molecular data is transforming the data into a form amenable to the application of learning algorithms. This process is broadly called "featurization" and involves turning a molecule into a vector or tensor of some sort. There are a number of different ways of doing that, and the choice of featurization is often dependent on the problem at hand. We have already seen two such methods: molecular fingerprints, and `ConvMol` objects for use with graph convolutions. In this tutorial we will look at some of the others.

## Colab

This tutorial and the rest in this sequence can be done in Google colab. If you'd like to open this notebook in colab, you can use the following link.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepchem/deepchem/blob/master/examples/tutorials/Going_Deeper_on_Molecular_Featurizations.ipynb)



In [ ]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

## Featurizers

In DeepChem, a method of featurizing a molecule (or any other sort of input) is defined by a `Featurizer` object.  There are three different ways of using featurizers.

1. When using the MoleculeNet loader functions, you simply pass the name of the featurization method to use.  We have seen examples of this in earlier tutorials, such as `featurizer='ECFP'` or `featurizer='GraphConv'`.

2. You also can create a Featurizer and directly apply it to molecules.  For example:

In [1]:
import deepchem as dc

featurizer = dc.feat.CircularFingerprint()
print(featurizer(['CC', 'CCC', 'CCO']))

Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (C:\Users\cex\anaconda3\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [7]:
featurizer = dc.feat.CircularFingerprint()
print(featurizer(['CN1C=NC2=C1C(=O)N(C(=O)N2C)C', 'CCC', 'CCO']))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


3. When creating a new dataset with the DataLoader framework, you can specify a Featurizer to use for processing the data.  We will see this in a future tutorial.

We use propane (CH<sub>3</sub>CH<sub>2</sub>CH<sub>3</sub>, represented by the SMILES string `'CCC'`) as a running example throughout this tutorial. Many of the featurization methods use conformers of the molecules. A conformer can be generated using the `ConformerGenerator` class in `deepchem.utils.conformers`. 

### RDKitDescriptors

In [10]:
import rdkit

`RDKitDescriptors` featurizes a molecule by using RDKit to compute values for a list of descriptors. These are basic physical and chemical properties: molecular weight, polar surface area, numbers of hydrogen bond donors and acceptors, etc. This is most useful for predicting things that depend on these high level properties rather than on detailed molecular structure.

Intrinsic to the featurizer is a set of allowed descriptors, which can be accessed using `RDKitDescriptors.allowedDescriptors`. The featurizer uses the descriptors in `rdkit.Chem.Descriptors.descList`, checks if they are in the list of allowed descriptors, and computes the descriptor value for the molecule.

Let's print the values of the first ten descriptors for propane.

In [11]:
RDKitDescriptors.allowedDescriptors

NameError: name 'RDKitDescriptors' is not defined

In [15]:
rdkit_featurizer = dc.feat.RDKitDescriptors()
features = rdkit_featurizer(['CN1C=NC2=C1C(=O)N(C(=O)N2C)C'])[0]
for feature, descriptor in zip(features[:], rdkit_featurizer.descriptors):
    print(descriptor, feature)

MaxAbsEStateIndex 11.672037037037038
MaxEStateIndex 11.672037037037038
MinAbsEStateIndex 0.31657407407407345
MinEStateIndex -0.3599999999999992
qed 0.5384628262372215
MolWt 194.194
HeavyAtomMolWt 184.11399999999998
ExactMolWt 194.08037556
NumValenceElectrons 74.0
NumRadicalElectrons 0.0
MaxPartialCharge 0.3317034326359889
MinPartialCharge -0.3278634660994821
MaxAbsPartialCharge 0.3317034326359889
MinAbsPartialCharge 0.3278634660994821
FpDensityMorgan1 1.1428571428571428
FpDensityMorgan2 1.7857142857142858
FpDensityMorgan3 2.4285714285714284
BCUT2D_MWHI 16.161282315860735
BCUT2D_MWLOW 10.48818798692214
BCUT2D_CHGHI 2.110255216429423
BCUT2D_CHGLO -2.02151749706659
BCUT2D_LOGPHI 2.0118198978857027
BCUT2D_LOGPLOW -2.369665405920737
BCUT2D_MRHI 5.691604588780562
BCUT2D_MRLOW 0.7054356050886157
AvgIpc 2.53868764707295
BalabanJ 3.221281081084589
BertzCT 616.5285919942219
Chi0 10.455665446700477
Chi0n 8.182701232117186
Chi0v 8.182701232117186
Chi1 6.5365812602091875
Chi1n 4.107927642957933
Chi

Of course, there are many more descriptors than this.

In [13]:
print('The number of descriptors present is: ', len(features))

The number of descriptors present is:  209


### WeaveFeaturizer and MolGraphConvFeaturizer

We previously looked at graph convolutions, which use `ConvMolFeaturizer` to convert molecules into `ConvMol` objects.  Graph convolutions are a special case of a large class of architectures that represent molecules as graphs.  They work in similar ways but vary in the details.  For example, they may associate data vectors with the atoms, the bonds connecting them, or both.  They may use a variety of techniques to calculate new data vectors from those in the previous layer, and a variety of techniques to compute molecule level properties at the end.

DeepChem supports lots of different graph based models.  Some of them require molecules to be featurized in slightly different ways.  Because of this, there are two other featurizers called `WeaveFeaturizer` and `MolGraphConvFeaturizer`.  They each convert molecules into a different type of Python object that is used by particular models.  When using any graph based model, just check the documentation to see what featurizer you need to use with it.

### CoulombMatrix

All the models we have looked at so far consider only the intrinsic properties of a molecule: the list of atoms that compose it and the bonds connecting them.  When working with flexible molecules, you may also want to consider the different conformations the molecule can take on.  For example, when a drug molecule binds to a protein, the strength of the binding depends on specific interactions between pairs of atoms.  To predict binding strength, you probably want to consider a variety of possible conformations and use a model that takes them into account when making predictions.

The Coulomb matrix is one popular featurization for molecular conformations.  Recall that the electrostatic Coulomb interaction between two charges is proportional to $q_1 q_2/r$ where $q_1$ and $q_2$ are the charges and $r$ is the distance between them.  For a molecule with $N$ atoms, the Coulomb matrix is a $N \times N$ matrix where each element gives the strength of the electrostatic interaction between two atoms.  It contains information both about the charges on the atoms and the distances between them.  More information on the functional forms used can be found [here](https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.108.058301).

To apply this featurizer, we first need a set of conformations for the molecule.  We can use the `ConformerGenerator` class to do this.  It takes a RDKit molecule, generates a set of energy minimized conformers, and prunes the set to only include ones that are significantly different from each other.  Let's try running it for propane.

In [25]:
from rdkit import Chem

generator = dc.utils.ConformerGenerator(max_conformers=5)
propane_mol = generator.generate_conformers(Chem.MolFromSmiles('CCC'))
print("Number of available conformers for propane: ", len(propane_mol.GetConformers()))

Number of available conformers for propane:  1


It only found a single conformer.  This shouldn't be surprising, since propane is a very small molecule with hardly any flexibility.  Let's try adding another carbon.

In [19]:
butane_mol = generator.generate_conformers(Chem.MolFromSmiles('CCCC'))
print("Number of available conformers for butane: ", len(butane_mol.GetConformers()))

Number of available conformers for butane:  3


In [20]:
caffeine_mol = generator.generate_conformers(Chem.MolFromSmiles('CN1C=NC2=C1C(=O)N(C(=O)N2C)C'))
print("Number of available conformers for caffeine: ", len(caffeine_mol.GetConformers()))

Number of available conformers for caffeine:  1


Now we can create a Coulomb matrix for our molecule.

In [28]:
coulomb_mat = dc.feat.CoulombMatrix(max_atoms=20)
features = coulomb_mat(caffeine_mol)
print(features)

Failed to featurize datapoint 0, [H]c1nc2c(c(=O)n(C([H])([H])[H])c(=O)n2C([H])([H])[H])n1C([H])([H])[H]. Appending empty array
Exception message: 


[]


Notice that many elements are 0.  To combine multiple molecules in a batch we need all the Coulomb matrices to be the same size, even if the molecules have different numbers of atoms.  We specified `max_atoms=20`, so the returned matrix  has size (20, 20).  The molecule only has 11 atoms, so only an 11 by 11 submatrix is nonzero.

### CoulombMatrixEig

An important feature of Coulomb matrices is that they are invariant to molecular rotation and translation, since the interatomic distances and atomic numbers do not change.  Respecting symmetries like this makes learning easier.  Rotating a molecule does not change its physical properties.  If the featurization does change, then the model is forced to learn that rotations are not important, but if the featurization is invariant then the model gets this property automatically.

Coulomb matrices are not invariant under another important symmetry: permutations of the atoms' indices.  A molecule's physical properties do not depend on which atom we call "atom 1", but the Coulomb matrix does.  To deal with this, the `CoulumbMatrixEig` featurizer was introduced, which uses the eigenvalue spectrum of the Coulumb matrix and is invariant to random permutations of the atom's indices.  The disadvantage of this featurization is that it contains much less information ($N$ eigenvalues instead of an $N \times N$ matrix), so models will be more limited in what they can learn.

`CoulombMatrixEig` inherits from `CoulombMatrix` and featurizes a molecule by first computing the Coulomb matrices for different conformers of the molecule and then computing the eigenvalues for each Coulomb matrix. These eigenvalues are then padded to account for variation in number of atoms across molecules.

In [29]:
coulomb_mat_eig = dc.feat.CoulombMatrixEig(max_atoms=20)
features = coulomb_mat_eig(propane_mol)
print(features)

[[60.07620352 29.62963132 22.75497753  0.57137859  0.28781334  0.28548338
   0.27558183  0.18163796  0.17460996  0.17059717  0.166401    0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]]


# Congratulations! Time to join the Community!

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with DeepChem, we encourage you to finish the rest of the tutorials in this series. You can also help the DeepChem community in the following ways:

## Star DeepChem on [GitHub](https://github.com/deepchem/deepchem)
This helps build awareness of the DeepChem project and the tools for open source drug discovery that we're trying to build.

## Join the DeepChem Gitter
The DeepChem [Gitter](https://gitter.im/deepchem/Lobby) hosts a number of scientists, developers, and enthusiasts interested in deep learning for the life sciences. Join the conversation!

## Citing This Tutorial
If you found this tutorial useful please consider citing it using the provided BibTeX. 

In [ ]:
@manual{Intro7, 
 title={Going Deeper on Molecular Featurizations}, 
 organization={DeepChem},
 author={Ramsundar, Bharath}, 
 howpublished = {\url{https://github.com/deepchem/deepchem/blob/master/examples/tutorials/Going_Deeper_on_Molecular_Featurizations.ipynb}}, 
 year={2021}, 
} 